In [22]:
from ortools.linear_solver import pywraplp
import numpy as np
import cvxopt

def solve_linear_inequalities_maximize(A, b, c):
    """
    Solves the system of linear inequalities Ax < b while maximizing c^T x 
    using Google OR-Tools.

    Args:
    A (list of list of floats): Coefficient matrix
    b (list of floats): Right-hand side vector
    c (list of floats): Coefficient vector for the objective function

    Returns:
    solution (list of floats): An optimal solution to maximize c^T x subject to Ax < b if one exists, otherwise None
    """
    # Number of variables and constraints
    num_vars = len(A[0])
    num_constraints = len(b)

    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Create variables
    x = [solver.NumVar(-solver.infinity(), solver.infinity(), f'x{i}') for i in range(num_vars)]

    # Add constraints: Ax < b
    for i in range(num_constraints):
        constraint = solver.RowConstraint(-solver.infinity(), b[i], '')
        for j in range(num_vars):
            constraint.SetCoefficient(x[j], A[i][j])

    # Define the objective function: maximize c^T x
    objective = solver.Objective()
    for i, xi in enumerate(x):
        objective.SetCoefficient(xi, c[i])
    objective.SetMaximization()

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # If a solution exists, return the solution
        return [xi.solution_value() for xi in x]
    else:
        # If the problem has no solution
        return None


def solve_quadratic_program_cvxopt(Q, c, A, b):
    """
    Solves a quadratic programming problem using cvxopt: 
    Minimize (1/2) x^T Q x + c^T x subject to Ax >= b.

    Args:
    Q (numpy.ndarray): Coefficient matrix for the quadratic term.
    c (numpy.ndarray): Coefficient vector for the linear term.
    A (numpy.ndarray): Coefficient matrix for the constraints.
    b (numpy.ndarray): Right-hand side vector for the constraints.

    Returns:
    numpy.ndarray: Optimal solution vector x, if a solution exists. None otherwise.
    """
    Q = cvxopt.matrix(Q, tc='d')
    c = cvxopt.matrix(c, (len(c), 1), tc='d')  # Reshape c as a column matrix
    A = cvxopt.matrix(A, tc='d')
    b = cvxopt.matrix(b, tc='d')

    # Convert A and b to fit cvxopt format (Gx <= h)
    # Since our constraints are Ax >= b, we use G = -A and h = -b
    G = -A
    h = -b

    # Solve the quadratic programming problem
    sol = cvxopt.solvers.qp(Q, c, G, h)

    if 'optimal' not in sol['status']:
        return None
    return np.array(sol['x']).flatten()

# Example usage
Q = np.array([[2, 0], [0, 2]])
c = np.array([1, 1])
A = np.array([[1, 1], [-1, 0], [0, -1]])
b = np.array([-10, 0, 0])  # Adjusted to represent x1 + x2 >= 10 and x1, x2 >= 0

solution = solve_quadratic_program_cvxopt(Q, c, A, b)
print("Solution:", solution)



     pcost       dcost       gap    pres   dres
 0:  5.2800e+00 -1.9680e+01  2e+01  0e+00  9e-16
 1:  2.8623e-01 -2.6402e+00  3e+00  5e-17  1e-15
 2: -4.6226e-01 -8.3739e-01  4e-01  1e-17  2e-16
 3: -4.9986e-01 -5.1705e-01  2e-02  1e-16  5e-18
 4: -5.0000e-01 -5.0018e-01  2e-04  4e-16  8e-17
 5: -5.0000e-01 -5.0000e-01  2e-06  7e-17  1e-16
 6: -5.0000e-01 -5.0000e-01  2e-08  1e-16  6e-19
Optimal solution found.
Solution: [-0.50000001 -0.50000001]


In [19]:
!pip uninstall cvxpy

Found existing installation: cvxpy 1.1.18
Uninstalling cvxpy-1.1.18:
  Would remove:
    /home/mohamed/miniconda3/lib/python3.9/site-packages/_cvxcore.cpython-39-x86_64-linux-gnu.so
    /home/mohamed/miniconda3/lib/python3.9/site-packages/cvxpy-1.1.18.dist-info/*
    /home/mohamed/miniconda3/lib/python3.9/site-packages/cvxpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [15]:
A = [[1, 2], [3,3]]
b = [14, 35]
c = [3, 4]

solution = solve_linear_inequalities_maximize(A, b, c)
print("Solution:", solution)


Solution: [9.333333333333332, 2.333333333333334]
